Add all the code cells to train the the different loss functions and then compare them: 

In [ ]:

loss_fn_name = ["only_bce","combined_loss_p2_norm","combined_loss_dice"]


layers_front = [0,0,15]
layers_back = [25,65,85]
epoch_ranges = [3,3,3] 

loss_functions_f1_scores = [[],[],[]]
loss_functions_loss_scores = [[],[],[]]

for loss_fn_idx in range(3):
    for layer_idx in range(len(layers_front)):
        epoch_range = epoch_ranges[layer_idx]
        for epoch_counter in range(epoch_range):
            l_front = layers_front[layer_idx]
            l_back = layers_back[layer_idx]
            
            model_description = f"model_{loss_fn_idx+1}_mlp_decoder_finetune_last_{l_front}_{l_back}_epoch_{epoch_counter}"
            model_path = "custom_segment_anything/model_checkpoints/loss_finetuned/"+model_description
            
            model = load_sam_model("mlp",device,encoder_finetune_num_first_layers=0,encoder_finetune_num_last_layers=0,finetuned_model_name=model_path,sam_checkpoint_or_finetuned="finetuned",load_from_compiled=False)
            
            # for each model compute the f1 and loss score of the original dataset
            model.eval()
            
            with torch.no_grad():
                l_acc = 0.0
                score_acc = 0.0
                for image,mask in tqdm(gmaps_dataloader):
                    image = image.to(device)
                    mask = mask.to(device)
                    pred = model(image)
                    # compute loss and f1 score: 
                    score = mean_f1_score_from_logits(pred,mask)    
                    loss =  loss_fn(pred,mask)
                    # update running loss and f1 score
                    score_acc += score.item()
                    l_acc  += loss.item()
                    # store the loss and f1 score
                    
                mean_f1 = score_acc/len(gmaps_dataloader)
                mean_loss = l_acc/len(gmaps_dataloader)
                loss_functions_f1_scores[loss_fn_idx].append(mean_f1)
                loss_functions_loss_scores[loss_fn_idx].append(mean_loss)
                print(f"Loss Function {loss_fn_idx}: Layer_idx: {layer_idx} Epoch: {epoch_counter}, (test) Loss: {l_acc/len(gmaps_dataloader)}, F1-Score: {score_acc/len(gmaps_dataloader)}")
            
# plot the resolts for all loss function in two seperate plots, one for f1 and one for loss



In [ ]:
import matplotlib.pyplot as plt
epoch_sum = sum(epoch_ranges)
epochs = list(range(epoch_sum))

# Plot F1 Scores
plt.figure(figsize=(12, 6))
for idx in range(3):
    plt.plot(epochs, loss_functions_f1_scores[idx], label=loss_fn_name[idx])
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('F1 Score over Epochs for Different Loss Functions')
plt.legend()
plt.show()

# Plot Loss Scores
plt.figure(figsize=(12, 6))
for idx in range(3):
    plt.plot(epochs, loss_functions_loss_scores[idx], label=loss_fn_name[idx])
plt.xlabel('Epoch')
plt.ylabel('weighted BCE+Dice Loss')
plt.title('Loss over Epochs for Different Loss Functions')
plt.legend()
plt.show()